In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

/home/mike/anaconda3/envs/nlp/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/mike/anaconda3/envs/nlp/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/mike/anaconda3/envs/nlp/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/mike/anaconda3/envs/nlp/lib/python3.6/site-packages/ten

In [2]:
pickle_file = './notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (100000, 28, 28) (100000,)
Validation set (5000, 28, 28) (5000,)
Test set (5000, 28, 28) (5000,)


In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (100000, 28, 28, 1) (100000, 10)
Validation set (5000, 28, 28, 1) (5000, 10)
Test set (5000, 28, 28, 1) (5000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))/ predictions.shape[0])

## Project

In [60]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64
learning_rate = 0.05

graph = tf.Graph()

with graph.as_default():
    # Input data
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels= tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size * 4, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    # Model
    with tf.name_scope('cnn'):
        def model(data):
            conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
            hidden = tf.nn.relu(conv + layer1_biases)
    #         print(hidden.shape)
            conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
            hidden = tf.nn.relu(conv + layer2_biases)
    #         print(hidden.shape)
            shape = hidden.get_shape().as_list()
            reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
            hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
            return tf.matmul(hidden, layer4_weights) + layer4_biases
    
    ## Training computation
    with tf.name_scope("loss"):
        logits = model(tf_train_dataset)
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    ## Gradient
    with tf.name_scope("train"):
        optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    
    ## Prediction for the training, valid and test
    with tf.name_scope("eval"):
        training_prediction = tf.nn.softmax(logits)
        valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
        test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [61]:
num_steps = 1001
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, training_prediction], feed_dict=feed_dict)
        if step % 100 == 0:
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.239974
Minibatch accuracy: 25.0%
Validation accuracy: 15.4%
Minibatch loss at step 100: 1.179944
Minibatch accuracy: 62.5%
Validation accuracy: 72.3%
Minibatch loss at step 200: 1.007976
Minibatch accuracy: 81.2%
Validation accuracy: 79.2%
Minibatch loss at step 300: 0.606713
Minibatch accuracy: 81.2%
Validation accuracy: 80.2%
Minibatch loss at step 400: 1.008331
Minibatch accuracy: 62.5%
Validation accuracy: 80.1%
Minibatch loss at step 500: 0.553356
Minibatch accuracy: 87.5%
Validation accuracy: 82.5%
Minibatch loss at step 600: 0.824569
Minibatch accuracy: 81.2%
Validation accuracy: 81.8%
Minibatch loss at step 700: 0.622564
Minibatch accuracy: 81.2%
Validation accuracy: 82.4%
Minibatch loss at step 800: 0.696274
Minibatch accuracy: 87.5%
Validation accuracy: 82.3%
Minibatch loss at step 900: 0.418111
Minibatch accuracy: 81.2%
Validation accuracy: 83.6%
Minibatch loss at step 1000: 0.359053
Minibatch accuracy: 93.8%
Validation accuracy: 83.7%

## Problem 1

In [5]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64
learning_rate = 0.05

graph = tf.Graph()

with graph.as_default():
    # Input data
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels= tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset[:100, :])
    tf_test_dataset = tf.constant(test_dataset[:100, :])
    
    # Variables
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    # Model
    with tf.name_scope('cnn'):
        def model(data):
            conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
            hidden = tf.nn.relu(conv + layer1_biases)
            
            ## Adding pooling
            hidden = tf.nn.max_pool(hidden, [1,2,2,1], [1,2,2,1], padding='SAME')
            
            conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
            hidden = tf.nn.relu(conv + layer2_biases)
            
            ## Adding pooling
            hidden = tf.nn.max_pool(hidden, [1,2,2,1], [1,2,2,1], padding='SAME')
            
            shape = hidden.get_shape().as_list()
            reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
            hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
            return tf.matmul(hidden, layer4_weights) + layer4_biases
    
    ## Training computation
    with tf.name_scope("loss"):
        logits = model(tf_train_dataset)
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    ## Gradient
    with tf.name_scope("train"):
        optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    
    ## Prediction for the training, valid and test
    with tf.name_scope("eval"):
        training_prediction = tf.nn.softmax(logits)
        valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
        test_prediction = tf.nn.softmax(model(tf_test_dataset))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [6]:
num_steps = 1001
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, training_prediction], feed_dict=feed_dict)
        if step % 100 == 0:
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels[:100, :]))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels[:100,:]))

Initialized
Minibatch loss at step 0: 4.653955
Minibatch accuracy: 18.8%
Validation accuracy: 7.0%
Minibatch loss at step 100: 1.908216
Minibatch accuracy: 25.0%
Validation accuracy: 29.0%
Minibatch loss at step 200: 1.170637
Minibatch accuracy: 62.5%
Validation accuracy: 77.0%
Minibatch loss at step 300: 0.665242
Minibatch accuracy: 87.5%
Validation accuracy: 82.0%
Minibatch loss at step 400: 1.154830
Minibatch accuracy: 62.5%
Validation accuracy: 76.0%
Minibatch loss at step 500: 0.379103
Minibatch accuracy: 93.8%
Validation accuracy: 82.0%
Minibatch loss at step 600: 0.874010
Minibatch accuracy: 87.5%
Validation accuracy: 83.0%
Minibatch loss at step 700: 0.582474
Minibatch accuracy: 81.2%
Validation accuracy: 85.0%
Minibatch loss at step 800: 0.790524
Minibatch accuracy: 75.0%
Validation accuracy: 87.0%
Minibatch loss at step 900: 0.387903
Minibatch accuracy: 81.2%
Validation accuracy: 83.0%
Minibatch loss at step 1000: 0.369732
Minibatch accuracy: 87.5%
Validation accuracy: 85.0%


## Problem 2

In [9]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 100
learning_rate = 0.01
keep_prob = 0.9
decay_steps = 100
decay_rate = 0.99

graph = tf.Graph()

with graph.as_default():
    # Input data
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels= tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # learning rate decay
    global_step = tf.Variable(0, trainable=False)
    learning_rate_decay = tf.train.exponential_decay(learning_rate, global_step, decay_steps, decay_rate)
    
    # Variables
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size * 4, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    # Model
    with tf.name_scope('cnn'):
        def model(data):
            conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
            hidden = tf.nn.relu(conv + layer1_biases)
            lrn = tf.nn.local
            conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
            hidden = tf.nn.relu(conv + layer2_biases)
            shape = hidden.get_shape().as_list()
            reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
            
            # dropout
#             reshape = tf.nn.dropout(reshape, keep_prob)
            
            hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
            return tf.matmul(hidden, layer4_weights) + layer4_biases
    
    ## Training computation
    with tf.name_scope("loss"):
        logits = model(tf_train_dataset)
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    ## Gradient
    with tf.name_scope("train"):
        optimizer = tf.train.GradientDescentOptimizer(learning_rate_decay).minimize(loss, global_step=global_step)
    
    ## Prediction for the training, valid and test
    with tf.name_scope("eval"):
        training_prediction = tf.nn.softmax(logits)
        valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
        test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [10]:
num_steps = 1001
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, training_prediction], feed_dict=feed_dict)
        if step % 100 == 0:
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.229975
Minibatch accuracy: 18.8%
Validation accuracy: 10.0%
Minibatch loss at step 100: 1.342831
Minibatch accuracy: 50.0%
Validation accuracy: 57.4%
Minibatch loss at step 200: 1.085020
Minibatch accuracy: 68.8%
Validation accuracy: 75.0%
Minibatch loss at step 300: 0.598252
Minibatch accuracy: 87.5%
Validation accuracy: 76.8%
Minibatch loss at step 400: 1.268827
Minibatch accuracy: 68.8%
Validation accuracy: 77.4%
Minibatch loss at step 500: 0.661358
Minibatch accuracy: 87.5%
Validation accuracy: 79.3%
Minibatch loss at step 600: 0.799740
Minibatch accuracy: 87.5%
Validation accuracy: 79.4%
Minibatch loss at step 700: 0.521679
Minibatch accuracy: 87.5%
Validation accuracy: 79.7%
Minibatch loss at step 800: 0.615414
Minibatch accuracy: 81.2%
Validation accuracy: 80.9%
Minibatch loss at step 900: 0.654740
Minibatch accuracy: 81.2%
Validation accuracy: 81.4%
Minibatch loss at step 1000: 0.310858
Minibatch accuracy: 87.5%
Validation accuracy: 81.6%